In [68]:
import pandas as pd
from hnmchallenge.data_reader import DataReader
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from hnmchallenge.dataset import Dataset
from hnmchallenge.filtered_dataset import FilterdDataset
from hnmchallenge.models.top_pop import TopPop
from hnmchallenge.evaluation.python_evaluation import map_at_k
from hnmchallenge.constant import *
from hnmchallenge.models.sgmc.sgmc import SGMC
from hnmchallenge.models.ease.ease import EASE
from hnmchallenge.models.itemknn.itemknn import ItemKNN 
from hnmchallenge.data_reader import DataReader
import datetime


In [69]:
dataset = FilterdDataset()
dr = DataReader()

In [70]:
user_data = dr.get_filtered_full_data()

In [71]:
user_data["t_dat"].max()

Timestamp('2020-09-22 00:00:00')

In [72]:
last_month = user_data[(user_data["t_dat"]>="2020-09-1")]

In [73]:
last_month[DEFAULT_USER_COL].nunique()

189031

In [74]:
last_month

,t_dat,customer_id,article_id,price,sales_channel_id
13569706,2020-09-01,90837,6097,0.013542,1
13569707,2020-09-01,90837,5824,0.018627,1
13569708,2020-09-01,90837,16527,0.012695,1
13569709,2020-09-01,90837,17047,0.016932,1
13569710,2020-09-01,1114723,16739,0.033881,1
...,...,...,...,...,...
14361083,2020-09-22,198005,20308,0.059305,2
14361084,2020-09-22,198005,16094,0.042356,2
14361085,2020-09-22,798833,21707,0.043203,1
14361086,2020-09-22,395317,4994,0.006763,1


In [75]:
sorted_data = last_month.sort_values([DEFAULT_USER_COL, "t_dat"]).reset_index(drop=True)

In [76]:
sorted_data

,t_dat,customer_id,article_id,price,sales_channel_id
0,2020-09-10,8,17388,0.016932,1
1,2020-09-20,15,21368,0.016271,2
2,2020-09-20,15,8694,0.011390,2
3,2020-09-20,15,19362,0.065102,2
4,2020-09-20,15,19395,0.032559,2
...,...,...,...,...,...
791377,2020-09-22,1136205,7580,0.022017,2
791378,2020-09-22,1136205,1765,0.016932,2
791379,2020-09-22,1136205,838,0.032186,2
791380,2020-09-22,1136205,12955,0.022017,2


In [77]:
sorted_data[DEFAULT_USER_COL].nunique()

189031

In [78]:
sorted_data["last_buy"] = sorted_data.groupby(DEFAULT_USER_COL)["t_dat"].transform(max)

In [79]:
hold_out=sorted_data[sorted_data["t_dat"]==sorted_data["last_buy"]]

In [80]:
hold_out

,t_dat,customer_id,article_id,price,sales_channel_id,last_buy
0,2020-09-10,8,17388,0.016932,1,2020-09-10
1,2020-09-20,15,21368,0.016271,2,2020-09-20
2,2020-09-20,15,8694,0.011390,2,2020-09-20
3,2020-09-20,15,19362,0.065102,2,2020-09-20
4,2020-09-20,15,19395,0.032559,2,2020-09-20
...,...,...,...,...,...,...
791377,2020-09-22,1136205,7580,0.022017,2,2020-09-22
791378,2020-09-22,1136205,1765,0.016932,2,2020-09-22
791379,2020-09-22,1136205,838,0.032186,2,2020-09-22
791380,2020-09-22,1136205,12955,0.022017,2,2020-09-22


In [81]:
hold_out[DEFAULT_USER_COL].nunique()

189031

In [82]:
hold_in1=sorted_data[sorted_data["t_dat"]!=sorted_data["last_buy"]]

In [83]:
hold_in2 = user_data[(user_data["t_dat"]<"2020-09-1")]

In [84]:
hold_in=pd.concat([hold_in1,hold_in2],axis=0)

In [85]:
hold_in

,t_dat,customer_id,article_id,price,sales_channel_id,last_buy
9,2020-09-01,23,2873,0.025407,2,2020-09-20
10,2020-09-01,23,16951,0.025407,2,2020-09-20
11,2020-09-01,23,6749,0.025407,2,2020-09-20
12,2020-09-01,23,14886,0.016932,2,2020-09-20
13,2020-09-01,23,3215,0.016932,2,2020-09-20
...,...,...,...,...,...,...
13569701,2020-08-31,137561,1806,0.050831,1,NaT
13569702,2020-08-31,446503,17459,0.025407,2,NaT
13569703,2020-08-31,446503,17283,0.033881,2,NaT
13569704,2020-08-31,554399,18562,0.025407,1,NaT


In [86]:
hold_in=hold_in.drop("last_buy",axis=1)

In [87]:
hold_out=hold_out.drop("last_buy",axis=1)

In [88]:
hold_in

,t_dat,customer_id,article_id,price,sales_channel_id
9,2020-09-01,23,2873,0.025407,2
10,2020-09-01,23,16951,0.025407,2
11,2020-09-01,23,6749,0.025407,2
12,2020-09-01,23,14886,0.016932,2
13,2020-09-01,23,3215,0.016932,2
...,...,...,...,...,...
13569701,2020-08-31,137561,1806,0.050831,1
13569702,2020-08-31,446503,17459,0.025407,2
13569703,2020-08-31,446503,17283,0.033881,2
13569704,2020-08-31,554399,18562,0.025407,1


In [89]:
hold_in = hold_in.sort_values(by=[DEFAULT_USER_COL,'t_dat'],ignore_index=True)

In [90]:
hold_in

,t_dat,customer_id,article_id,price,sales_channel_id
0,2018-09-20,0,0,0.008458,2
1,2018-09-20,0,0,0.008458,2
2,2019-08-05,0,0,0.007610,2
3,2019-08-16,0,644,0.039576,2
4,2019-10-06,0,1952,0.030492,2
...,...,...,...,...,...
13781276,2020-09-21,1135011,14,0.013797,1
13781277,2020-09-21,1135011,130,0.012186,1
13781278,2020-09-21,1135134,16232,0.033881,1
13781279,2020-09-21,1135134,12175,0.042356,1


In [91]:
hold_out

,t_dat,customer_id,article_id,price,sales_channel_id
0,2020-09-10,8,17388,0.016932,1
1,2020-09-20,15,21368,0.016271,2
2,2020-09-20,15,8694,0.011390,2
3,2020-09-20,15,19362,0.065102,2
4,2020-09-20,15,19395,0.032559,2
...,...,...,...,...,...
791377,2020-09-22,1136205,7580,0.022017,2
791378,2020-09-22,1136205,1765,0.016932,2
791379,2020-09-22,1136205,838,0.032186,2
791380,2020-09-22,1136205,12955,0.022017,2


In [92]:
hold_in.reset_index(drop=True).to_feather(dr.get_preprocessed_data_path() / "last_day_holdin.feather")
hold_out.reset_index(drop=True).to_feather(dr.get_preprocessed_data_path() / "last_day_holdout.feather")

In [93]:
hold_out[DEFAULT_USER_COL].nunique()

189031

In [94]:
hold_in[DEFAULT_USER_COL].nunique()

1116980